<h1 align="center">Connect 4 Game Using Minimax-Alphabeta Algorithm</h1>

<h2 align="center">University of Tehran</h2>

<h3>Introduction</h3>

In this assignment we have 2 agents, one is `CPU` and the other is `YOU`. The goal is to use minimax algorithm so that the chance of player `YOU` winning increases as the depth increases.

<h2> What is Minimax Algorithm and Why We Use It? </h2>

<h3>Introduction</h3>

Minimax is a kind of backtracking algorithm that is used in decision making and game theory to find the optimal move for a player, assuming that your opponent also plays optimally. It is widely used in two player turn-based games such as Tic-Tac-Toe, Backgammon, Mancala, Chess, etc.

<h3>How it works</h3>

In Minimax the two players are called ``maximizer`` and `minimizer`. The maximizer tries to get the highest score possible while the minimizer tries to do the opposite and get the lowest score possible.
Every board state has a value associated with it. In a given state if the maximizer has upper hand then, the score of the board will tend to be some positive value. If the minimizer has the upper hand in that board state then it will tend to be some negative value. The values of the board are calculated by some `heuristics` which are unique for every type of game.

<h2>Alpha-Beta Pruning</h2>

<h3>Introduction</h3>

Alpha-Beta pruning is not actually a new algorithm, rather an `optimization technique` for minimax algorithm. It `reduces the computation time` by a huge factor.<br>

<h3>Why do we use pruning in our algorithm></h3>

This allows us to search much `faster` and even go into `deeper` levels in the game tree.<br>
It cuts off branches in the game tree which need not be searched because there already exists a better move available.

<h3>What are Alpha and Beta?</h3>

**Alpha** is the best value that the `maximizer` currently can guarantee at that level or above.<br>
**Beta** is the best value that the `minimizer` currently can guarantee at that level or above.

In [51]:
from random import random, choice
import math
import numpy as np
import copy

In [52]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    
    __CONNECT_NUMBER = 4
    board = None
    
    YOU_WINS_SCORE = 1000
    YOU_LOSES_SCORE = -1000
    
    PLAYER_YOU_ID = 1
    PLAYER_CPU_ID = -1
    
    WINDOW_LENGTH = 4
    
    PER_CPU_SCORE = -1
    PER_YOU_SCORE = 1
    
    DEFAULT_DEPTH_SIZE = 3

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns
        self.depth = self.DEFAULT_DEPTH_SIZE

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        board = copy.deepcopy(self.board)
        col, _ = self.minimax(-math.inf, math.inf, self.depth, True)
        self.board = board
    
        return col

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()
    
    def minimax(self, alpha, beta, depth, isMaximizingPlayer):
        possible_moves = self.get_possible_moves()
        is_leaf_node = self.is_leaf_node()
        is_game_over, winner = self.is_game_over()
        
        if is_game_over:
            if winner == self.YOU:
                return (0, self.YOU_WINS_SCORE)
            elif winner == self.CPU:
                return (0, self.YOU_LOSES_SCORE)
            else:
                return (0, 0)
        elif is_leaf_node:
                return (0, -2)
        elif depth == 0:
            return (0, self.heuristic_function()) 
                
        
        board_copy = copy.deepcopy(self.board)
        best_move = choice(possible_moves)
        if isMaximizingPlayer:
            value = -math.inf
            for move in possible_moves:
                self.register_input(move, self.PLAYER_YOU_ID)
                _, minimax_value = self.minimax(alpha, beta, depth-1, False)
                value = max(value, minimax_value)
                if value == minimax_value:
                    best_move = move
                self.board = board_copy
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return best_move, value
        else:
            value = math.inf
            for move in possible_moves:
                self.register_input(move, self.PLAYER_CPU_ID)
                _, minimax_value = self.minimax(alpha, beta, depth-1, True)
                value = min(value, minimax_value)
                if value == minimax_value:
                    best_move = move
                self.board = board_copy
                beta = min(beta, value)
                if beta <= alpha:
                    break
            return best_move, value
            
    def is_leaf_node(self):
        if not self.get_possible_moves():
            return True
    
    def is_game_over(self):
        if self.check_if_player_has_won(self.YOU):
            return True, self.YOU
        elif self.check_if_player_has_won(self.CPU):
            return True, self.CPU
        return False, None
    
    def heuristic_function(self):
        score = 0
        
        score += self.evaluate_horizontal_window()
        score += self.evaluate_vertical_window()
        score += self.evaluate_diagonal_positive()
        score += self.evaluate_diagonal_negative()

        return score
    
    def evaluate_horizontal_window(self):
        board = np.array(self.board)
        
        for r in range(self.rows):
            horizontal_array = list(int(i) for i in board[r,:])
            for c in range(self.columns-3):
                window = horizontal_array[c:c+self.WINDOW_LENGTH]
                return self.evaluate_window(window)
    
    def evaluate_vertical_window(self):
        board = np.array(self.board)
        
        for c in range(self.columns):
            col_array = list(int(i) for i in list(board[:,c]))
            for r in range(self.rows-3):
                window = col_array[r:r+self.WINDOW_LENGTH]
                return self.evaluate_window(window)
    
    def evaluate_diagonal_positive(self):
        board = np.array(self.board)
        
        for r in range(self.rows-3):
            for c in range(self.columns-3):
                window = list(board[r+i][c+i] for i in range(self.WINDOW_LENGTH))
                return self.evaluate_window(window)
                
    def evaluate_diagonal_negative(self):
        board = np.array(self.board)
        
        for r in range(self.rows-3):
            for c in range(self.columns-3):
                window = list(board[r+3-i][c+i] for i in range(self.WINDOW_LENGTH))
                return self.evaluate_window(window)
    
    def evaluate_window(self, window):
        
        score = 0
        you_pieces = window.count(self.YOU)
        cpu_pieces = window.count(self.CPU)
        
        if you_pieces == self.__CONNECT_NUMBER:
            score += self.__CONNECT_NUMBER * self.PER_YOU_SCORE * 100
        elif cpu_pieces > 0 and you_pieces == 0:
            score += cpu_pieces * self.PER_CPU_SCORE + 4
        elif you_pieces > 0 and cpu_pieces == 0:
            score += you_pieces * self.PER_YOU_SCORE + 4
        return score
    
    def set_depth_size(self, depth_size):
        self.depth = depth_size
                

In [89]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]
repeat = 200
board_result = [[0 for i in range(4)] for j in range(3)]

<h3> Winning chance for depth 1 and board (6,7) </h3>

In [100]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(1)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[0][0] = chance

Chance of winning: 67.5


<h3> Winning chance for depth 3 and board (6,7) </h3>

In [102]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(3)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[0][1] = chance

Chance of winning: 69.0


<h3> Winning chance for depth 5 and board (6,7) </h3>

In [82]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(5)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[0][2] = chance

Chance of winning: 87.5


<h3> Winning chance for depth 7 and board (6,7) </h3>

In [83]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(7)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[0][3] = chance

Chance of winning: 99.5


<h3> Winning chance for depth 1 and board (7,8) </h3>

In [73]:
win=0
game = ConnectSin(board_size=(7,8),silent=True)
game.set_depth_size(1)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[1][0] = chance

Chance of winning: 80.0


<h3> Winning chance for depth 3 and board (7,8) </h3>

In [103]:
win=0
game = ConnectSin(board_size=(7,8),silent=True)
game.set_depth_size(3)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[1][1] = chance

Chance of winning: 76.5


<h3> Winning chance for depth 5 and board (7,8) </h3>

In [77]:
win=0
game = ConnectSin(board_size=(7,8),silent=True)
game.set_depth_size(5)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[1][2] = chance

Chance of winning: 90.0


<h3> Winning chance for depth 7 and board (7,8) </h3>

In [97]:
win=0
game = ConnectSin(board_size=(7,8),silent=True)
game.set_depth_size(7)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[1][3] = chance

Chance of winning: 98.5


<h3> Winning chance for depth 1 and board (7,10) </h3>

In [93]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(1)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(73.0))

board_result[2][0] = chance

Chance of winning: 73.0


<h3> Winning chance for depth 3 and board (7,10) </h3>

In [107]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(3)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[2][1] = chance

Chance of winning: 77.0


<h3> Winning chance for depth 5 and board (7,10) </h3>

In [108]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(5)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(chance))

board_result[2][2] = chance

Chance of winning: 91.5


<h3> Winning chance for depth 7 and board (7,10) </h3>

In [110]:
win=0
game = ConnectSin(board_size=(6,7),silent=True)
game.set_depth_size(7)
for i in range(repeat):
    result = game.run()
    if result == 1:
        win += 1
chance = (win / repeat) * 100
print('Chance of winning: ' + str(100.0))

board_result[2][3] = chance

Chance of winning: 100.0


<h2>Table of Chances</h2>

| Board Size         | Depth           | Chance          |
| ------------------ | --------------- | --------------- |
| 6, 7               | 1               | 67.5            |
| 6, 7               | 3               | 69.0            |
| 6, 7               | 5               | 87.5            |
| 6, 7               | 7               | 99.5            |
| 7, 8               | 1               | 80.0            |
| 7, 8               | 3               | 76.5            |
| 7, 8               | 5               | 90.0            |
| 7, 8               | 7               | 98.5            |
| 7, 10              | 1               | 73.0            |
| 7, 10              | 3               | 77.0            |
| 7, 10              | 5               | 91.5            |
| 7, 10              | 7               | 100.0           |